# Visual Exploratory analysis of Telecom churn data

## 1. Dataset

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'svg'

In [ ]:
df = pd.read_csv('../input/telecom_churn.csv')
df.head()

In [ ]:
df.info();

The last column 'churn' is our target variable.

# 2. Univariate visualization

Univariate analysis looks at one feature at a time. when we analyse a feature independently, we are usually interested of its values and ignore other features in the dataset.

### 2.1 Quantitative features:

Quantitative features take on ordered numerical values. These values can be discrete ( i.e integers), or continuous ( i.e real numbers) and usually express a count or measurement.

### Histograms 

In [ ]:
features = ['total day minutes', 'total intl calls']
df[features].hist(figsize=(10,4));

A histogram groups values into bins of equal value range. The shape of the histogram provides the clues about the underlying distribution type. We can also spot any skewness in tis shape.  Knowing a distribution of the feature values is importtant when we use machine leaning methods that assume a particular type of it, most often Gaussian.

In the above plot 'total day minutes' is normally distributed, but the 'total intl calls' is skewed right ( itis tail is longer on the right )

## Density plots

Another ways to understand the distribution is by using the density plots or more formally Kernal Density Plots. These are like a smoothed version of histogram.

In [ ]:
df[features].plot(kind='density', subplots=True, layout=(1,2), 
                  sharex=False, figsize=(10,4));

We can also pot distribution with seaborn's distplot().  distplot() displays the histogram with kernel density estimate on top. Let us see the distribution of 'ttal day minutes'.

In [ ]:
x = df['total intl calls']
sns.distplot(x);

The height of the histogram bars here is normed and shows the density rather than the number of examles in each bin.

### Box plot

In [ ]:
sns.boxplot(x = 'total intl calls', data = df);

Let's see how to interpret a box plot. Its components are a box (obviously, this is why it is called a box plot), the so-called whiskers, and a number of individual points (outliers).

The box by itself illustrates the interquartile spread of the distribution; its length is determined by the 25th(Q1) and 75th(Q3) percentiles. The vertical line inside the box marks the median (50%) of the distribution.

The whiskers are the lines extending from the box. They represent the entire scatter of data points, specifically the points that fall within the interval (Q1−1.5⋅IQR,Q3+1.5⋅IQR), where IQR=Q3−Q1 is the interquartile range.

Outliers that fall out of the range bounded by the whiskers are plotted individually as black points along the central axis.

We can see that a large number of international calls is quite rare in our data.

### Violin plot

violinplot is also another distribution plot.

violinplot displays the kernal density estimate on both sides

In [ ]:
_, axes = plt.subplots(1,2, sharey=True, figsize=(6,4))
sns.boxplot(data=df['total intl calls'], ax=axes[0]);
sns.violinplot(data=df['total intl calls'], ax = axes[1]);

The difference between the box and violin plots is that the former illustrates certain statistics concerning individual examples in a dataset while the violin plot concentrates more on the smoothed distribution as a whole.

In our case, the violin plot does not contribute any additional information about the data as everything is clear from the box plot alone.

### Numerical statisticks of the distribution

In [ ]:
df[features].describe()

### 2.2 Categorical and Binary features

Categorical features take on a fixed number of values. Each of these values assigns an observation to a corresponding group, known as a category, which reflects some qualitative property of this example. Binary variables are an important special case of categorical variables when the number of possible values is exactly 2. If the values of a categorical variable are ordered, it is called ordinal.

### Frequency table

Let us check the distribution of our target varible (i.e churn )
First we will print frequency table, which shows how freqent each value of the categorical variable is. 

In [ ]:
df['churn'].value_counts()

By default, the entries in the output are sorted from the most to the least frequently-occurring values.

In our case, the data is not balanced; that is, our two target classes, loyal and disloyal customers, are not represented equally in the dataset. Only a small part of the clients canceled their subscription to the telecom service.

### Bar plot

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,2))

sns.countplot(x='churn', data = df, ax = axes[0])
sns.countplot(x='customer service calls', data = df, ax = axes[1]);

We have seen the histograms above, now we have seen the bar plots, they look similar, but there are several differences between them:

1. Histograms are best suited for looking at the distribution of numerical variables while bar plots are used for categorical features.
2. The values on the X-axis in the histogram are numerical; a bar plot can have any type of values on the X-axis: numbers, strings, booleans
3. The histogram's X-axis is a Cartesian coordinate axis along which values cannot be changed; the ordering of the bars is not predefined. Still, it is useful to note that the bars are often sorted by height, that is, the frequency of the values. Also, we we consider ordinal varibles ( like cusotmer service calls in our data), the bars are usually ordered by variable value.

The left chart above vividly illustrates the imbalance in our target variable. The bar plot for Customer service calls on the right gives a hint that the majority of customers resolve their problems in maximum 2–3 calls. But, as we want to be able to predict the minority class, we may be more interested in how the fewer dissatisfied customers behave. It may well be that the tail of that bar plot contains most of our churn. These are just hypotheses for now, so let's move on to some more interesting and powerful visual techniques

# 3. Multivariate visualization

Multivariate plots allow us to see relationships between two and more different variables, all in one figure. Just as in the case of univariate plots, the specific type of visualization will depend on the types of the variables being analyzed.

## 3.1 Quantitative-Quantitative

### Correlation matrix

We will analyse the correlations among the numerical variables in our dataset.
Let us drop the non-numerical variables and print the correlation matrix

In [ ]:
numerical = list(set(df.columns)-set(['state', 'international plan', 'voice mail plan',
                                     'area code', 'churn', 'customer service calls']))
corr_matrix = df[numerical].corr()
corr_matrix

let us plot the heatmap

In [ ]:
sns.heatmap(corr_matrix);

As seen in the correlation plot above, we can notice that there are 4 variables ( 'total day charge', 'total eve charge', 'total night charge', 'total intl charge' ) are directly depended on ('total day call', 'total eve calls', 'total night calls', 'total intl calls'). 
These are called dependent variables and can be leftout since they do not contribute any additional information. so let's drop them:

In [ ]:
numerical = list(set(numerical)-set(['total day charge', 'total eve charge', 'total night charge', 'total intl charge']))
corr_matrix = df[numerical].corr()
sns.heatmap(corr_matrix);

### Scatter plot

The scatter plot displays values of two numerical variables as Cartesian coordinates in 2D. Scatter plots in 3D are also possible.

In [ ]:
plt.scatter(df['total day minutes'], df['total night minutes']);

The above plot is not interesting, these features are non-correlated, because the ellipse like shape is aligned with the axes.

Let us use the seaborn's jointplot

In [ ]:
sns.jointplot(x='total day minutes', y='total night minutes', data=df, kind = 'scatter');

The jointplot plots two histograms that may be useful in some cases.
Using the jointplot, we can also get a smoothed version of our bivariate distribution.

In [ ]:
sns.jointplot('total day minutes', 'total night minutes', data = df, kind = 'kde', color='g');

this is nothing but a bivariate version of Kernal Density Plot discussed earlier.

### Scatterplot matrix

In [ ]:
# pairplot() may become very slow with the SVG format
%config InlineBackend.figure_format = 'png'
sns.pairplot(df[numerical]);

In [ ]:
%config InlineBackend.figure_format = 'svg'

### 3.2 Quantitative - Categorical

we will try to gain new insights for churn prediction from the interactions between the numerical and categorical features.

we will try to interpret how the input variables are related to the target varible 'churn'

Previously, you learned about scatter plots. Additionally, their points can be color or size coded so that the values of a third categorical variable are also presented in the same figure. We can achieve this with the scatter() function seen above, but, let's try a new function called lmplot() and use the parameter hue to indicate our categorical feature of interest:

In [ ]:
sns.lmplot('total day minutes', 'total night minutes', data = df, hue='churn', fit_reg=False);

It seems that our small proportion of disloyal customers lean towards the top-right corner; that is, such customers tend to spend more time on the phone during both day and night. But this is not absolutely clear, and we won't make any definitive conclusions from this chart.

Now, let’s create box plots to visualize the distribution statistics of the numerical variables in two disjoint groups: the loyal customers (churn=False) and those who left (churn=True).

In [ ]:
numerical = list(set(numerical)-set(['phone number']))

In [ ]:
# sometimes we can analyse an ordinal variables just as numerical one
numerical.append('customer service calls')

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(10,7))
for idx, feat in enumerate(numerical):
    ax = axes[int(idx / 4), idx % 4]
    sns.boxplot(x='churn', y = feat, data=df, ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel(feat)
fig.tight_layout();

From the above charts, we can notice that there is a lot of discrepancy in distribution between the two groups is for three variables, 'total day minutes', 'cusotmer service calls' and 'number vmail messages'. 

Let's look at the distribution of day minutes spoken for the loyal and disloyal cusotmers separately. We will create box and violin plots for 'total day minutes' grouped by the target variable.

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

sns.boxplot(x='churn', y='total day minutes', data = df, ax = axes[0]);
sns.violinplot(x='churn', y='total day minutes', data = df, ax = axes[1]);

In this case, the violin plot does not contribute any additional information about our data as everything is clear from the box plot alone: disloyal customers tend to talk on the phone more.

An interesting observation: on average, customers that discontinue their contracts are more active users of communication services. Perhaps they are unhappy with the tariffs, so a possible measure to prevent churn could be a reduction in call rates. The company will need to undertake additional economic analysis to find out whether such measures would be beneficial.

When we want to analyze a quantitative variable in two categorical dimensions at once, there is a suitable function for this in the seaborn library called catplot(). For example, let's visualize the interaction between total day minutes and two categorical variables in the same plot:

In [ ]:
sns.catplot(x='churn', y = 'total day minutes', col='customer service calls',
           data=df[df['customer service calls'] < 8], kind = 'box', col_wrap=4, height=3, aspect=0.8);

From the above plots, we can notice that, starting with 4 calls, total day minutes may no longer be the main factor for customer churn. Perhaps, in addition to ur previous guess about the tariffs, there are cusomters that are dissatisfied with the service due to other problems, which might lead to fewer number of day minutes spent on calls.

### 3.3 Categorical-Categorical

As we saw earlier in this article, the variable Customer service calls has few unique values and, thus, can be considered either numerical or ordinal. We have already seen its distribution with a count plot. Now, we are interested in the relationship between this ordinal feature and the target variable Churn.

Let's look at the distribution of the number of calls to the customer service, again using a count plot. This time, let's also pass the parameter hue=Churn that adds a categorical dimension to the plot:

In [ ]:
sns.countplot(x='customer service calls', hue='churn', data=df);

An observation: the churn rate increases significantly after 4 or more calls to the customer service.

Now, let's look at the relationship between Churn and the binary features, International plan and Voice mail plan.

In [ ]:
_, axes = plt.subplots(1, 2, sharey = True, figsize=(10,4))

sns.countplot(x = 'international plan', hue = 'churn', data = df, ax = axes[0]);
sns.countplot(x = 'voice mail plan', hue = 'churn', data = df, ax = axes[1]);

An observation: when International Plan is enabled, the churn rate is much higher; the usage of the international plan by the customer is a strong feature. We do not observe the same effect with Voice mail plan.

### Contingency table

In addition to using graphical means for categorical analysis, there is a traditional tool from statistics: a contingency table, also called a cross tabulation. It represents multivariate frequency distribution of categorical variables in tabular form. In particular, it allows us to see the distribution of one variable conditional on the other by looking along a column or row.

Let's try to see how Churn is related to the categorical variable State by creating a cross tabulation

In [ ]:
pd.crosstab(df['state'], df['churn'])

In the case of State, the number of distinct values is rather high: 51. We see that there are only a few data points available for each individual state – only 3 to 17 customers in each state abandoned the operator. Let's ignore that for a second and calculate the churn rate for each state, sorting it from high to low:

In [ ]:
df.groupby(['state'])['churn'].agg([np.mean]).sort_values(by='mean', ascending=False)

It seems that the churn rate in New Jersey and California are above 25% and less than 6% for Hawaii and Alaska. However, these conclusions are based on too few examples, and our observation could be a mere property of our particular dataset. 

## 4. Whole dataset

### 4.1 Naive approach

we have seen the different faces of this dataset by guessing features and selecting a small number of them at a time for visualizaton. 

We could use hist() or create a scatterplot matrix with pairplot() for the whole dataset to look at all of our features simultaneously. But, when the number of features is high enough, this kind of visual analysis quickly becomes slow and inefficient. Besides, we would still be analyzing our variables in a pairwise fashion, not all at once.

### 4.2  Dimensionality reduction

Most real-world datasets have many features, sometimes, many thousands of them. Each of them can be considered as a dimension in the space of data points. Consequently, more often than not, we deal with high-dimensional datasets, where entire visualization is quite hard.

To look at a dataset as a whole, we need to decrease the number of dimensions used in visualization without losing much information about data. This task is called dimensionality reduction and is an example of an unsupervised learning problem because we need to derive new, low-dimensional features from the data itself, without any supervised input.

One of the well-known dimensionality reduction methods is Principal Component Analysis (PCA). Its limitation is that it is a linear algorithm that implies certain restrictions on the data.

There are also many non-linear methods, collectively called Manifold Learning. One of the best-known of them is t-SNE

### 4.3 t-SNE 

The name of the method looks complex and a bit intimidating: t-distributed Stohastic Neighbor Embedding. 

Its basic idea is simple: find a projection for a high-dimensional feature space onto a plane (or a 3D hyperplane, but it is almost always 2D) such that those points that were far apart in the initial n-dimensional space will end up far apart on the plane. Those that were originally close would remain close to each other.

Essentially, neighbor embedding is a search for a new and less-dimensional data representation that preserves neighborship of examples.

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

we will leave out the state and churn features and convert the values Yes/No of the binary features into numerical values using pandas.Series.map():

In [ ]:
X = df.drop(['churn', 'state', 'phone number'], axis = 1)
X['international plan'] = X['international plan'].map({'yes':1, 'no':0})
X['voice mail plan'] = X['voice mail plan'].map({'yes':1,'no':0})

In [ ]:
X.head()

We also need to normalize the data. For this, we will subtract the mean from each variable and divide it by its standard deviation. All of this can be done with StandardScaler.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Let us build t-SNE representation:

In [ ]:
%%time
tsne = TSNE(random_state=17)
tsne_repr = tsne.fit_transform(X_scaled)

In [ ]:
plt.scatter(tsne_repr[:,0], tsne_repr[:,1], alpha=0.5);

Let’s color this t-SNE representation according to the churn (blue for loyal customers, and orange for those who churned).

In [ ]:
plt.scatter(tsne_repr[:,0], tsne_repr[:,1], c=df['churn'].map({False:'blue', True:'orange'}), alpha=0.5);

We can see that customers who churned are concentrated in a few areas of the lower dimensional feature space.

To better understand the picture, we can also color it with the remaining binary features: International Plan and Voicemail. Orange dots here indicate instances that are positive for the corresponding binary feature.

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(12,5))

for i, name in enumerate(['international plan', 'voice mail plan']):
    axes[i].scatter(tsne_repr[:,0], tsne_repr[:,1],
                c=df[name].map({'yes' : 'orange', 'no':'blue'}), alpha=0.5);
    axes[i].set_title(name);

From the above plots, many dissatisfied customers who canceled their subscription are crowded together in one cluster representing the people with the international plan but no voice mail.